#### AlexNet
    AlexNet网络诞生于2012年，当年ImageNet竞赛的冠军，Top5错误率为16.4%
    Alex Krizhevsky, Ilya Sutskever, Geoffrey E. Hinton. ImageNet Classification with Deep Convolutional NeuralNetworks. In NIPS, 2012.
    
    结构：五层卷积，三层全连接
    输入：32*32*3
    C（核：96*3*3， 步长：1，填充：valid ）
    B（Yes，LRN*）
    A（relu）
    P（max，核：3*3，步长：2 ）
    D（None）
    
    C（核：256*3*3，步长：1，填充：valid ）
    B（Yes，LRN*）
    A（relu）
    P（max，核：3*3，步长：2 ）
    D（None）
    
    C（核：384*3*3，步长：1，填充：same ）
    B（None）
    A（relu）
    P（None）
    D（None）
    
    C（核：384*3*3，步长：1，填充：same ）
    B（None）
    A（relu）
    P（None）
    D（None）
    
    C（核：256*3*3，步长：1，填充：same ）
    B（None）
    A（relu）
    P（ max，核：3*3，步长：2 ）
    D（None）
    
    Dense（神经元：2048，激活：relu，Dropout：0.5）
    Dense（神经元：2048，激活：relu，Dropout：0.5）
    Dense（神经元：10，激活：softmax）

In [ ]:
import tensorflow as tf
import os
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Dropout, Flatten, Dense
from tensorflow.keras import Model

np.set_printoptions(threshold=np.inf)

cifar10 = tf.keras.datasets.cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0


class AlexNet8(Model):
    def __init__(self):
        super(AlexNet8, self).__init__()
###########################第一层################################
        self.c1 = Conv2D(filters=96, kernel_size=(3, 3))
        self.b1 = BatchNormalization()
        self.a1 = Activation('relu')
        self.p1 = MaxPool2D(pool_size=(3, 3), strides=2)
###########################第二层################################
        self.c2 = Conv2D(filters=256, kernel_size=(3, 3))
        self.b2 = BatchNormalization()
        self.a2 = Activation('relu')
        self.p2 = MaxPool2D(pool_size=(3, 3), strides=2)
###########################第三层################################
        self.c3 = Conv2D(filters=384, kernel_size=(3, 3), padding='same',
                         activation='relu')
###########################第四层################################                       
        self.c4 = Conv2D(filters=384, kernel_size=(3, 3), padding='same',
                         activation='relu')
###########################第五层################################                       
        self.c5 = Conv2D(filters=256, kernel_size=(3, 3), padding='same',
                         activation='relu')
        self.p3 = MaxPool2D(pool_size=(3, 3), strides=2)

        self.flatten = Flatten()
###########################第六层################################
        self.f1 = Dense(2048, activation='relu')
        self.d1 = Dropout(0.5)
###########################第七层################################
        self.f2 = Dense(2048, activation='relu')
        self.d2 = Dropout(0.5)
###########################第八层################################
        self.f3 = Dense(10, activation='softmax')

    def call(self, x):
        x = self.c1(x)
        x = self.b1(x)
        x = self.a1(x)
        x = self.p1(x)

        x = self.c2(x)
        x = self.b2(x)
        x = self.a2(x)
        x = self.p2(x)

        x = self.c3(x)

        x = self.c4(x)

        x = self.c5(x)
        x = self.p3(x)

        x = self.flatten(x)
        x = self.f1(x)
        x = self.d1(x)
        x = self.f2(x)
        x = self.d2(x)
        y = self.f3(x)
        return y


model = AlexNet8()

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['sparse_categorical_accuracy'])

checkpoint_save_path = "./checkpoint/AlexNet8.ckpt"
if os.path.exists(checkpoint_save_path + '.index'):
    print('-------------load the model-----------------')
    model.load_weights(checkpoint_save_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_save_path,
                                                 save_weights_only=True,
                                                 save_best_only=True)

history = model.fit(x_train, y_train, batch_size=32, epochs=5, validation_data=(x_test, y_test), validation_freq=1,
                    callbacks=[cp_callback])
model.summary()

# print(model.trainable_variables)
file = open('./weights.txt', 'w')
for v in model.trainable_variables:
    file.write(str(v.name) + '\n')
    file.write(str(v.shape) + '\n')
    file.write(str(v.numpy()) + '\n')
file.close()

###############################################    show   ###############################################

# 显示训练集和验证集的acc和loss曲线
acc = history.history['sparse_categorical_accuracy']
val_acc = history.history['val_sparse_categorical_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.subplot(1, 2, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()


Train on 50000 samples, validate on 10000 samples
Epoch 1/5
50000/50000 [==============================] - 1678s 34ms/sample - loss: 1.6251 - sparse_categorical_accuracy: 0.4033 - val_loss: 1.6622 - val_sparse_categorical_accuracy: 0.4142
Epoch 2/5
50000/50000 [==============================] - 1633s 33ms/sample - loss: 1.2898 - sparse_categorical_accuracy: 0.5446 - val_loss: 1.2542 - val_sparse_categorical_accuracy: 0.5636
Epoch 3/5
50000/50000 [==============================] - 1549s 31ms/sample - loss: 1.1480 - sparse_categorical_accuracy: 0.5991 - val_loss: 1.3304 - val_sparse_categorical_accuracy: 0.5310
Epoch 4/5
50000/50000 [==============================] - 1521s 30ms/sample - loss: 1.0718 - sparse_categorical_accuracy: 0.6285 - val_loss: 1.4420 - val_sparse_categorical_accuracy: 0.5262
Epoch 5/5
35456/50000 [====================>.........] - ETA: 6:58 - loss: 1.0089 - sparse_categorical_accuracy: 0.6513